In [ ]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [ ]:
import pickle
import numpy as np
np.set_printoptions(suppress=True)

# 1. Unbounded Computational Budget

### Loading the already saved dataset

In [ ]:
with open("39-bus Results/Final Results/1200_risky_data_list", "rb") as fp: 
    risky_data_list, M_episodes_qlearning = pickle.load(fp)

In [ ]:
checkRisky = np.array(risky_data_list)
print(checkRisky.shape)
print(checkRisky.shape[1], checkRisky.shape[2])

### Averaging the accumulated risk across the 25 MC iterations.

In [ ]:
np.mean(checkRisky[0, :, :], axis=0)[-1]

In [ ]:
np.std(checkRisky[0, :, :], axis=0)

In [ ]:
np.mean(checkRisky[1, :, :], axis=0)[-1]

In [ ]:
np.mean(checkRisky[2, :, :], axis=0)

## Finding the Range for Cumulative Regret Accuracy Metric

In [ ]:
from FaultChainSolver import FaultChainSovlerClass

selectCase = "39"   # "14, "30", "39", "118"
LOADING_FACTOR = 0.55
dataSetString = "loading055_39bus.h5"
M = 5  

FCsovler = FaultChainSovlerClass(LOADING_FACTOR, selectCase) # instantiated object!
dataSet = FCsovler.load_dataset(dataSetString)
FCsovler.print_datatset_information(dataSet)
riskyFaultChainDict = FCsovler.find_risky_fault_chains(dataSet, M)
riskyFaultChainDict_all = FCsovler.find_risky_fault_chains_all(dataSet)

In [ ]:
print( len(riskyFaultChainDict), len(dataSet) )
print( [len(riskyFaultChainDict) for riskyFaultChainDict in riskyFaultChainDict_all] )

### Finding the risk assocaited with the optimal S=1200 sequences.

In [ ]:
optimalSequenceRisk = np.sort(dataSet[:, 3])[::-1] # sorting the entire dataset in decreasing order
print( optimalSequenceRisk[:1200] )                # finding the 1200 most risky FC risks

In [ ]:
optimalSequenceRisk1200 = np.cumsum( optimalSequenceRisk[:1200] )
optimalSequenceRisk1200[-1]    # the sum of the entire sequences of risks

In [ ]:
optimalSequenceRisk1200[-1] - np.mean(checkRisky[0, :, :], axis=0)

In [ ]:
import matplotlib.pyplot as plt

x = optimalSequenceRisk1200[-1] - np.mean(checkRisky[0, :, :], axis=0)
y = optimalSequenceRisk1200[-1] - np.mean(checkRisky[1, :, :], axis=0)
z = optimalSequenceRisk1200[-1] - np.mean(checkRisky[2, :, :], axis=0)
a = optimalSequenceRisk1200[-1] - np.mean(checkRisky[3, :, :], axis=0)
b = optimalSequenceRisk1200[-1] - np.mean(checkRisky[4, :, :], axis=0)
plt.plot(x/1000)
plt.plot(y/1000)
plt.plot(z/1000)
plt.plot(a/1000)
plt.plot(b/1000)
plt.show()

In [ ]:
dataRegret = optimalSequenceRisk1200[-1]*np.ones((25, 1200)) - checkRisky
print(dataRegret.shape)

In [ ]:
from Evaluation import EvaluationClassIterationMCPlot
finalPlot = EvaluationClassIterationMCPlot([], [], [], [], [], [], [], [], [], [])
with open("39-bus Results/Final Results/1200_risky_data_list", "rb") as fp:
    risky_data_list, M_episodes_qlearning = pickle.load(fp)
#finalPlot.plot_regret(risky_data_list, M_episodes_qlearning)

## Finding the Range for Precision Accuracy Metric

In [ ]:
with open("39-bus Results/Final Results/1200_num_data_list", "rb") as fp:
    num_data_list, M_episodes_qlearning = pickle.load(fp)
    
checkNumdata = np.array(num_data_list)
checkNumdata.shape

In [ ]:
lengthArray = np.array( range(1, 1200+1) )
lengthArray[:10]

In [ ]:
precision = checkNumdata[0, :, :]/lengthArray
np.mean(precision, axis=0)

In [ ]:
precision = checkNumdata[1, :, :]/lengthArray
np.mean(precision, axis=0)

In [ ]:
precision = checkNumdata[2, :, :]/lengthArray
np.mean(precision, axis=0)

In [ ]:
precision = checkNumdata[3, :, :]/lengthArray
np.mean(precision, axis=0)

In [ ]:
precision = checkNumdata[4, :, :]/lengthArray
np.mean(precision, axis=0)

In [ ]:
import matplotlib.pyplot as plt

x = np.mean(checkNumdata[0, :, :]/lengthArray, axis=0)
y = np.mean(checkNumdata[1, :, :]/lengthArray, axis=0)
z = np.mean(checkNumdata[2, :, :]/lengthArray, axis=0)
a = np.mean(checkNumdata[3, :, :]/lengthArray, axis=0)
b = np.mean(checkNumdata[4, :, :]/lengthArray, axis=0)
pivot = 20
plt.plot(x[pivot:])
plt.plot(y[pivot:])
plt.plot(z[pivot:])
plt.plot(a[pivot:])
plt.plot(b[pivot:])
plt.show()

In [ ]:
from Evaluation import EvaluationClassIterationMCPlot
finalPlot = EvaluationClassIterationMCPlot([], [], [], [], [], [], [], [], [], [])
lengthArrayMatrix = np.vstack([lengthArray]*checkNumdata.shape[1])
dataPrecision = (checkNumdata/lengthArrayMatrix)
pivot = 6
#finalPlot.plot_precision(dataPrecision[:, :, pivot:], 1200-pivot)

In [ ]:
finalPlot.display_statistics_for_list(dataRegret, dataPrecision)

# 2. Bounded Computational Budget

In [ ]:
with open("39-bus Results/Final Results/time_risky_data_list", "rb") as fp:
    risky_data_list, M_episodes_qlearning = pickle.load(fp)
    
print( np.array(risky_data_list).shape, np.array(M_episodes_qlearning).shape)

In [ ]:
for approach in range(5):

    algorithm = risky_data_list[approach]
    MC_iterations = np.array(algorithm).shape[0]

    numberofFCsList, totalRiskList, regretList = [], [], []
    numRiskyFCs, precisionList = [], []
    
    for iteration in range(MC_iterations):
        temp = np.array( algorithm[iteration] )     # FC Risk, time taken
        numberofFCs, _ = temp.shape              
        #print(" Number of FCs: ", numberofFCs, " Total Risk: ", temp[:, 0][-1], " Optimal Sequence Risk: ", sum(optimalSequenceRisk[:numberofFCs]) )
        
        numberofFCsList.append(numberofFCs)
        cumu = temp[:, 0]
        normalArray = cumu.copy()
        normalArray[1:] = np.diff(cumu)
        #print( sum( normalArray > 34.3982*5 ) )
            
        numRiskyFCs.append( sum( normalArray >= 34*5 ) )
        precisionList.append( sum( normalArray >= 34*5 )/numberofFCs )
        
        totalRiskList.append( temp[:, 0][-1] )
        regretList.append( sum(optimalSequenceRisk[:numberofFCs]) - temp[:, 0][-1] )

    print("Avg No. of FCs found:         ", np.average( np.array(numberofFCsList) ))
    print("Range Total Risk found:       ", np.average( np.array(totalRiskList) ), "    +- " , np.std( np.array(totalRiskList) ), " also in percentage ", 100*np.std( np.array(totalRiskList) )/np.average( np.array(totalRiskList) ))
    print("Range Regret found:           ", np.average( np.array(regretList) ) ,   "    +- " , np.std( np.array(regretList) ), " also in percentage ", 100*np.std( np.array(regretList) )/np.average( np.array(regretList) ))
    print("Range No. of Risky FCs found: ", np.average( np.array(numRiskyFCs) ),   "    +- " , np.std( np.array(numRiskyFCs) ), " also in percentage ", 100*np.std( np.array(numRiskyFCs) )/np.average( np.array(numRiskyFCs) ))
    print("Range Precision found:        ", np.average( np.array(precisionList) ), "    +- " , np.std( np.array(precisionList) ), " also in percentage ", 100*np.std( np.array(precisionList) )/np.average( np.array(precisionList) ) )   
# for     print(" ")

In [ ]:
# for some reason, "num_data_list" stored here is not very helpful. I relied on "risky_data_list" in order to determine the number of risky FCs and precision as well.

with open("39-bus Results/Final Results/time_num_data_list", "rb") as fp:
    num_data_list, M_episodes_qlearning = pickle.load(fp)
    
print( np.array(num_data_list).shape, np.array(M_episodes_qlearning).shape)

In [ ]:
algorithm = num_data_list[4]
algorithm = np.array(algorithm)
algorithm = np.squeeze(algorithm)
algorithm.shape